# problem formulation
* The problem : Is model is able to predict to Whether a first date will lead to a relationship based on the profile of two people.
* we have 191 features as input to dataset and output is prediction of the probability of matching of two people (0-1, float) from column called match .the data mining function that we need in this problems is classification  predictions.
* The challenges on business understanding,data selection,data preparation as the data set is un balanced and  we have many of missing values.  Challenges according to real life  are difficult to match between two characters.

* **The impact of the problem** on real life helps us to solve social problem,  knows matching between two persons  before  actual relationship to avoid problems . so we can get to ideal solution  if  we could get  to a first date will lead to a relationship. So we apply life cycle of data mining to be able to predict Whether a first date will lead to a relationship.

Data collection.

Data selection.

Data preparation.

Modeling.

Evaluation.

Development.
Note: according to the problem ,I got to the ideal solution with xgb with tunning hyper paramter manually.

3.# The experimental protocol that I applied it .
* First: Applying oversampling to make data  in balance.
* Second: Using  hold out method that Indicates to spliting training data to train and test split to be able to measure performance.
* Using grid search ,randomsearch,bayes search for tunning hyper paramters and also using manual tunning.
# Preprocessing steps that I need:

**Try 1**:
* I used  pipline  to apply these steps (Preprocessing steps,model training ).

* **Preprocessing steps** .
* Handling missing value  with  Simple imputer class because data has alot of missing values.
* Scaling and Normlization with standard scalar class.
* one hot encoding: to convert categorical features to numeric features.

 **Try 2**:
* The same steps of try 1 in addition to dropping useless columns  and applying  feature selection with chi test

* Note : describtion of pipline in details later.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Data preparation

##Loading and Visualizing the Data
We first load the packages as usual.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler
from collections import Counter
import numpy as np
from sklearn .metrics import roc_auc_score
 
from sklearn.compose import ColumnTransformer
from sklearn.datasets import fetch_openml
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from xgboost.sklearn import XGBClassifier
from sklearn import metrics
from sklearn.feature_selection import SelectKBest, mutual_info_classif
import warnings


Next, we load the data

In [ ]:
df = pd.read_csv("/content/train.csv")
df

,gender,idg,condtn,wave,round,position,positin1,order,partner,pid,...,sinc3_3,intel3_3,fun3_3,amb3_3,attr5_3,sinc5_3,intel5_3,fun5_3,amb5_3,id
0,0,3,2,14,18,2,2.0,14,12,372.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2583
1,1,14,1,3,10,2,NaN,8,8,63.0,...,8.0,8.0,7.0,8.0,NaN,NaN,NaN,NaN,NaN,6830
2,1,14,1,13,10,8,8.0,10,10,331.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4840
3,1,38,2,9,20,18,13.0,6,7,200.0,...,9.0,8.0,8.0,6.0,NaN,NaN,NaN,NaN,NaN,5508
4,1,24,2,14,20,6,6.0,20,17,357.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4828
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5904,0,1,2,9,20,2,2.0,18,1,214.0,...,12.0,12.0,9.0,12.0,NaN,NaN,NaN,NaN,NaN,3390
5905,1,24,2,9,20,19,15.0,5,6,199.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4130
5906,0,13,2,11,21,5,5.0,3,18,290.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1178
5907,1,10,2,7,16,6,14.0,9,10,151.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5016


##Some of Data cleaning .


In [ ]:
df.isnull().sum().sort_values(ascending=False)# anything missing?

num_in_3    5449
numdat_3    4849
expnum      4627
amb7_2      4519
sinc7_2     4519
            ... 
position       0
round          0
wave           0
condtn         0
id             0
Length: 192, dtype: int64

In [ ]:
df=df.drop(labels='id', axis=1)# drop idt column (it  may not be useful)
# axis=1 indicates that we drop vertically

##Manipulating the Data
We can get the shape of the data, which indicates the number of samples and number of attributes


In [ ]:
df.info()# checking types:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5909 entries, 0 to 5908
Columns: 191 entries, gender to amb5_3
dtypes: float64(173), int64(10), object(8)
memory usage: 8.6+ MB


In [ ]:
df.head()

,gender,idg,condtn,wave,round,position,positin1,order,partner,pid,...,attr3_3,sinc3_3,intel3_3,fun3_3,amb3_3,attr5_3,sinc5_3,intel5_3,fun5_3,amb5_3
0,0,3,2,14,18,2,2.0,14,12,372.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,14,1,3,10,2,NaN,8,8,63.0,...,6.0,8.0,8.0,7.0,8.0,NaN,NaN,NaN,NaN,NaN
2,1,14,1,13,10,8,8.0,10,10,331.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,38,2,9,20,18,13.0,6,7,200.0,...,8.0,9.0,8.0,8.0,6.0,NaN,NaN,NaN,NaN,NaN
4,1,24,2,14,20,6,6.0,20,17,357.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df.shape# check shape

(5909, 191)

From  the info of data frame we have 8 objects so we can convert them to category to be able to apply one hot encoding later on.

In [ ]:
df2 = df.copy()# some simple transformation (always make a copy before you assign)

# categorical encoding of field
df2['field'] = df['field'].astype("category")

# categorical encoding of undergra
df2['undergra'] = df['undergra'].astype("category")
# categorical encoding of mn_sat
df2['mn_sat'] = df['mn_sat'].astype("category")
# categorical encoding of tuition
df2['tuition'] = df['tuition'].astype("category")
# categorical encoding of from
df2['from'] = df['from'].astype("category")
# categorical encoding of zipcode
df2['zipcode'] = df['zipcode'].astype("category")
# categorical encoding of income

df2['income'] = df['income'].astype("category")
# categorical encoding of career
df2['career'] = df['career'].astype("category")


In [ ]:
# lets have a look of the values:
df2

,gender,idg,condtn,wave,round,position,positin1,order,partner,pid,...,attr3_3,sinc3_3,intel3_3,fun3_3,amb3_3,attr5_3,sinc5_3,intel5_3,fun5_3,amb5_3
0,0,3,2,14,18,2,2.0,14,12,372.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,14,1,3,10,2,NaN,8,8,63.0,...,6.0,8.0,8.0,7.0,8.0,NaN,NaN,NaN,NaN,NaN
2,1,14,1,13,10,8,8.0,10,10,331.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,38,2,9,20,18,13.0,6,7,200.0,...,8.0,9.0,8.0,8.0,6.0,NaN,NaN,NaN,NaN,NaN
4,1,24,2,14,20,6,6.0,20,17,357.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5904,0,1,2,9,20,2,2.0,18,1,214.0,...,12.0,12.0,12.0,9.0,12.0,NaN,NaN,NaN,NaN,NaN
5905,1,24,2,9,20,19,15.0,5,6,199.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5906,0,13,2,11,21,5,5.0,3,18,290.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5907,1,10,2,7,16,6,14.0,9,10,151.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df2.info()#print data frame information.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5909 entries, 0 to 5908
Columns: 191 entries, gender to amb5_3
dtypes: category(8), float64(173), int64(10)
memory usage: 8.4 MB


In [ ]:



y = df2['match'] # target
X = df2.drop('match', axis=1) #features
print('original shape', X.shape, y.shape)




original shape (5909, 190) (5909,)


In [ ]:
df2['match'].value_counts()#check data is balanced or not

0    4921
1     988
Name: match, dtype: int64

Applying oversampling to make data  in balance.
Now we can apply approaches to random resampling for imbalanced classification.
they are oversampling and undersampling.I applied Random oversampling  that involves randomly selecting examples from the minority class, with replacement, and adding them to the training dataset. 

In [ ]:
oversample = RandomOverSampler(sampling_strategy='minority')# define oversampling strategy
# fit and apply the transform
X_over, y_over = oversample.fit_resample(X, y)

In [ ]:
# summarize class distribution
print(Counter(y_over))

Counter({0: 4921, 1: 4921})


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
  X_over,  y_over, test_size=0.33, random_state=42)#split the data


#plan for try 2
* I applied this try with xgb  to increase performance  of the model ,because  XGBoost converts weak learners to strong learners. this is actually second try  for me .
* According to  the problem he data is  clean  but we have many of missing value  so i used pipline to handle that and do some pre processing  steps (Tunable Pipeline).

**Try a**:
* I used Pipeline as it is useful ,there are often a fixed sequence of steps in processing the data.
* It is used to chain multiple estimators into one and hence, automate the machine learning process.
* the steps in  pipline in our case(I used it for hanle missing value as the data has many of missing values with simle imputer method.
* I used  it for applying one hot encoding to convert categorical to numerical.
* I used it for  for scaling data with standard scalar class.
* In this try I used Xgb classifier  and I know that it chooses important features  by default , i tuned hyper paramter manually.

* **Try b**:
* I used the same steps in addition to  tunning hyperparamter with grid search.
* **Ttry c**:
* I used the same steps in addition to  tunning hyperparamter with random search.

* **Try d:**
I used the same steps in addition  tunning hyperparamter with Bayesian Search.

#Observation :
* model with tunning  hyper paramter manually: I expect the best  because XGBoost has many tuning parameters and actually it is best private score on kaggle(0.88458 )
* model with tunning  hyper paramter with grid search: I expect it is less than  tunning hyperparamter manually because it takes alot of time(because it creates a model for every combination of the hyperparameter to find the best values ) and actually  private score of grid  on kaggle(0.86554 )
* model with tunning  hyper paramter with random search:I expect it is greater  than  tunning hyperparamter  with grid search because it is efficient(less trails) and  actually  private score of random  on kaggle(0.87895 )
* model with tunning  hyper paramter with Bayesian Search:I expect it is  greater  than  tunning hyperparamter  with random search  because it is faster
than it but it has   the same private score (0.87895)of random search.(because they select hyperparameters in an informed manner(By prioritizing hyperparameters that appear more promising from past results.



In [ ]:
# we extract numeric features and categorical features names
# for later use

# numeric features can be selected by: (based on the df2.info() output )
features_numeric = list(X_train.select_dtypes(include=['float64', 'int64']))

# categorical features can be selected by: (based on the df2.info() output )
features_categorical = list(X_train.select_dtypes(include=['category']))

print('numeric features:', features_numeric)


numeric features: ['gender', 'idg', 'condtn', 'wave', 'round', 'position', 'positin1', 'order', 'partner', 'pid', 'int_corr', 'samerace', 'age_o', 'race_o', 'pf_o_att', 'pf_o_sin', 'pf_o_int', 'pf_o_fun', 'pf_o_amb', 'pf_o_sha', 'attr_o', 'sinc_o', 'intel_o', 'fun_o', 'amb_o', 'shar_o', 'like_o', 'prob_o', 'met_o', 'age', 'field_cd', 'race', 'imprace', 'imprelig', 'goal', 'date', 'go_out', 'career_c', 'sports', 'tvsports', 'exercise', 'dining', 'museums', 'art', 'hiking', 'gaming', 'clubbing', 'reading', 'tv', 'theater', 'movies', 'concerts', 'music', 'shopping', 'yoga', 'exphappy', 'expnum', 'attr1_1', 'sinc1_1', 'intel1_1', 'fun1_1', 'amb1_1', 'shar1_1', 'attr4_1', 'sinc4_1', 'intel4_1', 'fun4_1', 'amb4_1', 'shar4_1', 'attr2_1', 'sinc2_1', 'intel2_1', 'fun2_1', 'amb2_1', 'shar2_1', 'attr3_1', 'sinc3_1', 'fun3_1', 'intel3_1', 'amb3_1', 'attr5_1', 'sinc5_1', 'intel5_1', 'fun5_1', 'amb5_1', 'attr', 'sinc', 'intel', 'fun', 'amb', 'shar', 'like', 'prob', 'met', 'match_es', 'attr1_s', 'sin

In [ ]:
print('category features', features_categorical)

category features ['field', 'undergra', 'mn_sat', 'tuition', 'from', 'zipcode', 'income', 'career']


define pipline

In [ ]:
# define a pipe line for numeric feature preprocessing
# we gave them a name so we can set their hyperparameters
transformer_numeric = Pipeline(
    steps=[
        ('imputer', SimpleImputer()),
        ('scaler', StandardScaler())]
)

# define a pipe line for categorical feature preprocessing
# we gave them a name so we can set their hyperparameters
transformer_categorical = Pipeline(
    steps=[
        ('imputer', SimpleImputer(strategy='constant')),
        ('onehot', OneHotEncoder(handle_unknown='ignore'))
    ]
)
# define the preprocessor 
# we gave them a name so we can set their hyperparameters
# we also specify what are the categorical 
preprocessor = ColumnTransformer(
    transformers=[
        ('num', transformer_numeric, features_numeric),
        ('cat', transformer_categorical,features_categorical )
    ]
)


**Build** model
* we can use pipline as sklearn model

In [ ]:
# combine the preprocessor with the model as a full tunable pipeline
# we gave them a name so we can set their hyperparameters
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy='constant')
full_pipline = Pipeline(
    steps=[
        ('preprocessor', preprocessor),
        
        ('my_classifier', 
           XGBClassifier(n_jobs=1, random_state=42, 
                                       subsample=0.8))#tunning hyperparamter manually
        
    ]
)
full_pipline

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['gender', 'idg', 'condtn',
                                                   'wave', 'round', 'position',
                                                   'positin1', 'order',
                                                   'partner', 'pid', 'int_corr',
                                                   'samerace', 'age_o',
                                                   'race_o', 'pf_o_att',
                                                   'pf_o_sin', 'pf_o_int',
                                                   

fit model

In [ ]:
full_pipline = full_pipline.fit(X_train, y_train)#fit model.

y_pred=full_pipline.predict_proba(X_test)
y_pred
# roc_auc_score(y_test, y_pred)
# auc = metrics.roc_auc_score(y_train, y_pred)
# y_train.shape
# y_pred.shape

array([[0.10853499, 0.891465  ],
       [0.18919861, 0.8108014 ],
       [0.92620057, 0.07379941],
       ...,
       [0.26919168, 0.7308083 ],
       [0.86690646, 0.13309355],
       [0.61717683, 0.38282317]], dtype=float32)

# load  test data and do the same pre processing.

In [ ]:
test_test = pd.read_csv("/content/test.csv")

In [ ]:
test_test.head()

,gender,idg,condtn,wave,round,position,positin1,order,partner,pid,...,sinc3_3,intel3_3,fun3_3,amb3_3,attr5_3,sinc5_3,intel5_3,fun5_3,amb5_3,id
0,0,5,2,2,16,3,NaN,13,13,52.0,...,7.0,8.0,6.0,8.0,NaN,NaN,NaN,NaN,NaN,934
1,0,33,2,14,18,6,6.0,4,8,368.0,...,8.0,7.0,7.0,8.0,6.0,7.0,6.0,5.0,5.0,6539
2,1,6,2,9,20,10,16.0,15,19,212.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6757
3,1,26,2,2,19,15,NaN,8,10,30.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2275
4,0,29,2,7,16,7,7.0,10,5,162.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1052


preprocessing

In [ ]:
df2_ts = test_test.copy()# some simple transformation (always make a copy before you assign)


# categorical encoding of field

df2_ts['field'] = test_test['field'].astype("category")

# categorical encoding of undergra
df2_ts['undergra'] = test_test['undergra'].astype("category")
# categorical encoding of mn_sat

df2_ts['mn_sat'] = test_test['mn_sat'].astype("category")
# categorical encoding of tuition'
df2_ts['tuition'] = test_test['tuition'].astype("category")
# categorical encoding of from
df2_ts['from'] = test_test['from'].astype("category")
# categorical encoding of zipcode
df2_ts['zipcode'] = test_test['zipcode'].astype("category")
# categorical encoding of income
df2_ts['income'] = test_test['income'].astype("category")
# categorical encoding of career
df2_ts['career'] = test_test['career'].astype("category")


In [ ]:
df2_ts = df2_ts.drop('id', axis=1)

In [ ]:
df2_ts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2469 entries, 0 to 2468
Columns: 190 entries, gender to amb5_3
dtypes: category(8), float64(173), int64(9)
memory usage: 3.5 MB


fit model

In [ ]:

full_pipline = full_pipline.fit(X, y)# fit model on all data

In [ ]:
test_pred=full_pipline.predict_proba(df2_ts)# prediction of model on all data

In [ ]:
test_pred

array([[0.964388  , 0.035612  ],
       [0.55213624, 0.44786376],
       [0.6687063 , 0.3312937 ],
       ...,
       [0.9408348 , 0.05916517],
       [0.9381759 , 0.0618241 ],
       [0.9572531 , 0.04274693]], dtype=float32)

Saving prediction in file .csv

In [ ]:
submission = pd.DataFrame()

submission['id'] = test_test['id']

submission['match'] = full_pipline.predict_proba(df2_ts)[:,1]

submission.to_csv('sample_submission_walkthrough123.csv', index=False)

#Try b: grid search
Grid Search with validation set

In [ ]:
param_grid = [{'my_classifier__n_estimators' : [90, 100, 110],
               'my_classifier__learning_rate': [0.1, 0.05, 0.01]}]
               #[900, 1000, 1100]
               #[1000, 1500, 2000]

In [ ]:
# param_grid = {
#     'preprocessor__num__imputer__strategy': ['most_frequent'],
#     # preprocessor__num__imputer__strategy points to preprocessor->num (a Pipeline)-> imputer -> strategy
#     'my_classifier__n_estimators': [20, 30, 40],  
#      # my_classifier__n_estimators points to my_classifier->n_estimators 
#     'my_classifier__max_depth':[10, 20, 30]       
# }

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.model_selection import PredefinedSplit

# Further split the original training set to a train and a validation set
X_train2, X_val, y_train2, y_val = train_test_split(
    X_train, y_train, train_size = 0.8, stratify = y_train, random_state = 2022)

# Create a list where train data indices are -1 and validation data indices are 0
# X_train2 (new training set), X_train
split_index = [-1 if x in X_train2.index else 0 for x in X_train.index]

# Use the list to create PredefinedSplit
pds = PredefinedSplit(test_fold = split_index)
# here we still use X_train; but the grid search model
# will use our predefined split internally to determine 
# which sample belongs to the validation set
grid_search = GridSearchCV(
    full_pipline, param_grid, cv=2, verbose=1, n_jobs=2, 
    scoring='roc_auc')

grid_search.fit(X_train, y_train)

print('best score {}'.format(grid_search.best_score_))
print('best score {}'.format(grid_search.best_params_))


Fitting 2 folds for each of 9 candidates, totalling 18 fits
best score 0.9168943902000777
best score {'my_classifier__learning_rate': 0.1, 'my_classifier__n_estimators': 110}


In [ ]:
grid_search.predict_proba(X_test)# prediction on x_test

array([[0.09732419, 0.9026758 ],
       [0.18915004, 0.81084996],
       [0.9411824 , 0.05881765],
       ...,
       [0.2726887 , 0.7273113 ],
       [0.85031843, 0.14968155],
       [0.63770634, 0.36229366]], dtype=float32)

In [ ]:
full_pipline = full_pipline.fit(X, y) #fit model on all data

In [ ]:
test_pred=full_pipline.predict_proba(df2_ts)# prediction on all data set.
test_pred

array([[0.964388  , 0.035612  ],
       [0.55213624, 0.44786376],
       [0.6687063 , 0.3312937 ],
       ...,
       [0.9408348 , 0.05916517],
       [0.9381759 , 0.0618241 ],
       [0.9572531 , 0.04274693]], dtype=float32)

Saving prediction in file .csv

In [ ]:
submission = pd.DataFrame()

submission['id'] = test_test['id']

submission['match'] = full_pipline.predict_proba(df2_ts)[:,1]

submission.to_csv('try223 afaf.csv', index=False)

Tryc: Random search

In [ ]:
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.model_selection import PredefinedSplit

grid_search = RandomizedSearchCV(
    full_pipline, param_grid, cv=pds, verbose=1, n_jobs=2, 
    # number of random trials
    n_iter=10,
    scoring='roc_auc')

grid_search.fit(X_train, y_train)

print('best score {}'.format(grid_search.best_score_))
print('best score {}'.format(grid_search.best_params_))

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_search.py:296: UserWarning: The total space of parameters 9 is smaller than n_iter=10. Running 9 iterations. For exhaustive searches, use GridSearchCV.
  UserWarning,


Fitting 1 folds for each of 9 candidates, totalling 9 fits
best score 0.9204603980428945
best score {'my_classifier__n_estimators': 110, 'my_classifier__learning_rate': 0.1}


In [ ]:
grid_search.predict_proba(X_test)

array([[0.09732419, 0.9026758 ],
       [0.18915004, 0.81084996],
       [0.9411824 , 0.05881765],
       ...,
       [0.2726887 , 0.7273113 ],
       [0.85031843, 0.14968155],
       [0.63770634, 0.36229366]], dtype=float32)

In [ ]:
full_pipline = full_pipline.fit(X, y)#fit model on all data

In [ ]:
test_pred=full_pipline.predict_proba(df2_ts)# prediction on all data set.
test_pred

array([[0.964388  , 0.035612  ],
       [0.55213624, 0.44786376],
       [0.6687063 , 0.3312937 ],
       ...,
       [0.9408348 , 0.05916517],
       [0.9381759 , 0.0618241 ],
       [0.9572531 , 0.04274693]], dtype=float32)

Saving prediction in file .csv

In [ ]:
submission = pd.DataFrame()

submission['id'] = test_test['id']

submission['match'] = full_pipline.predict_proba(df2_ts)[:,1]

submission.to_csv('try33afaf.csv', index=False)

In [ ]:
!pip install scikit_optimize

     |████████████████████████████████| 100 kB 4.6 MB/s 


Try d baysian

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import PredefinedSplit
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer



bayes_search = BayesSearchCV(
    full_pipline, param_grid, cv=pds, verbose=1, n_jobs=2, 
    # number of random trials
    n_iter=10,
    scoring='roc_auc')

bayes_search.fit(X_train, y_train)

print('best score {}'.format(bayes_search.best_score_))
print('best score {}'.format(bayes_search.best_params_))

Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
best score 0.919191222455211
best score OrderedDict([('my_classifier__learning_rate', 0.1), ('my_classifier__n_estimators', 100)])


In [ ]:
bayes_search.predict_proba(X_test)# prediction

array([[0.10853499, 0.891465  ],
       [0.18919861, 0.8108014 ],
       [0.92620057, 0.07379941],
       ...,
       [0.26919168, 0.7308083 ],
       [0.86690646, 0.13309355],
       [0.61717683, 0.38282317]], dtype=float32)

In [ ]:
full_pipline = full_pipline.fit(X, y)# fit model on all dataset.

In [ ]:
test_pred=full_pipline.predict_proba(df2_ts)# prediction
test_pred

array([[0.964388  , 0.035612  ],
       [0.55213624, 0.44786376],
       [0.6687063 , 0.3312937 ],
       ...,
       [0.9408348 , 0.05916517],
       [0.9381759 , 0.0618241 ],
       [0.9572531 , 0.04274693]], dtype=float32)

Saving prediction in file .csv

In [ ]:
submission = pd.DataFrame()

submission['id'] = test_test['id']

submission['match'] = full_pipline.predict_proba(df2_ts)[:,1]

submission.to_csv('try44 bayesafaf.csv', index=False)

#plan for try 1
* In this try :the same steps of pipline in addition to dropping useless columns  before  applying pipline and applying feature selection.
* I used different classifier ,different features, different preprocessing.
* I used random forest classifier , feature selection with Chi-square Test to find important features, and  I dropped  columns with  missing values more than 70%. 


**Try a**:

* In this try I used radom forest classifier  and I know that it chooses important features  by default , I tuned hyper paramter manually.

* **Try b**:
* I used the same steps in addition to  tunning hyperparamter with grid search.
* **Ttry d**:
* I used the same steps in addition to  tunning hyperparamter with random search.

* **Try e:**
I used the same steps in addition  tunning hyperparamter with Bayesian Search.

#Observation :
* I expect that  random forest with tunning hyper paramter manually is the best then  bayesian search, then random search then grid search but
it  has the same private score in kaggle(0.86554) with the 4 approaches.



##read data

In [ ]:
import pandas as pd
df = pd.read_csv("/content/train.csv")
df

,gender,idg,condtn,wave,round,position,positin1,order,partner,pid,...,sinc3_3,intel3_3,fun3_3,amb3_3,attr5_3,sinc5_3,intel5_3,fun5_3,amb5_3,id
0,0,3,2,14,18,2,2.0,14,12,372.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2583
1,1,14,1,3,10,2,NaN,8,8,63.0,...,8.0,8.0,7.0,8.0,NaN,NaN,NaN,NaN,NaN,6830
2,1,14,1,13,10,8,8.0,10,10,331.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4840
3,1,38,2,9,20,18,13.0,6,7,200.0,...,9.0,8.0,8.0,6.0,NaN,NaN,NaN,NaN,NaN,5508
4,1,24,2,14,20,6,6.0,20,17,357.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4828
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5904,0,1,2,9,20,2,2.0,18,1,214.0,...,12.0,12.0,9.0,12.0,NaN,NaN,NaN,NaN,NaN,3390
5905,1,24,2,9,20,19,15.0,5,6,199.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4130
5906,0,13,2,11,21,5,5.0,3,18,290.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1178
5907,1,10,2,7,16,6,14.0,9,10,151.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5016


Try a: pipline with tunning hyperparamter manually

#Data preparation

In [ ]:
df.isnull().sum().sort_values(ascending=False)#check null values

num_in_3    5449
numdat_3    4849
expnum      4627
amb7_2      4519
sinc7_2     4519
            ... 
position       0
round          0
wave           0
condtn         0
id             0
Length: 192, dtype: int64

In [ ]:
# def missing_values(df):
#    missing=pd.DataFrame(df.isnull().sum()/len(df))*100
#    missing.columns=['missing_values(%)']
#    missing['missing_values(numbers)']=pd.DataFrame(df.isnull().sum())
#    return missing.sort_values(by='missing_values(%)',ascending=False)


In [ ]:
def rmissingvaluecol(dff,threshold): #remove columns that has more than 70%  missing values.
    l = []
    l = list(dff.drop(dff.loc[:,list((100*(dff.isnull().sum()/len(dff.index))>=threshold))].columns, 1).columns.values)
    print("# Columns having more than %s percent missing values:"%threshold,(dff.shape[1] - len(l)))
    print("Columns:\n",list(set(list((dff.columns.values))) - set(l)))
    return l




In [ ]:
l = rmissingvaluecol(df,70)# apply function  on dataframe.
df1 = df[l]
df1 

# Columns having more than 70 percent missing values: 21
Columns:
 ['amb5_3', 'amb7_3', 'shar7_2', 'amb7_2', 'fun5_3', 'fun7_3', 'attr7_3', 'sinc7_2', 'sinc5_3', 'intel7_3', 'intel5_3', 'attr7_2', 'numdat_3', 'fun7_2', 'num_in_3', 'sinc7_3', 'shar2_3', 'shar7_3', 'intel7_2', 'attr5_3', 'expnum']


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


,gender,idg,condtn,wave,round,position,positin1,order,partner,pid,...,sinc2_3,intel2_3,fun2_3,amb2_3,attr3_3,sinc3_3,intel3_3,fun3_3,amb3_3,id
0,0,3,2,14,18,2,2.0,14,12,372.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2583
1,1,14,1,3,10,2,NaN,8,8,63.0,...,NaN,NaN,NaN,NaN,6.0,8.0,8.0,7.0,8.0,6830
2,1,14,1,13,10,8,8.0,10,10,331.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4840
3,1,38,2,9,20,18,13.0,6,7,200.0,...,8.0,8.0,8.0,8.0,8.0,9.0,8.0,8.0,6.0,5508
4,1,24,2,14,20,6,6.0,20,17,357.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4828
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5904,0,1,2,9,20,2,2.0,18,1,214.0,...,7.0,7.0,10.0,8.0,12.0,12.0,12.0,9.0,12.0,3390
5905,1,24,2,9,20,19,15.0,5,6,199.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4130
5906,0,13,2,11,21,5,5.0,3,18,290.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1178
5907,1,10,2,7,16,6,14.0,9,10,151.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5016


In [ ]:
# checking types:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5909 entries, 0 to 5908
Columns: 171 entries, gender to id
dtypes: float64(152), int64(11), object(8)
memory usage: 7.7+ MB


In [ ]:
df1=df1.drop(labels='id', axis=1)# drop id.

In [ ]:
def missing(dff):
    print (round((dff.isnull().sum() * 100/ len(dff)),2).sort_values(ascending=False))# print percentage of missing value on data frame.

missing(df1)

attr4_3     64.95
sinc2_3     64.95
sinc4_3     64.95
intel4_3    64.95
fun4_3      64.95
            ...  
order        0.00
partner      0.00
match        0.00
samerace     0.00
gender       0.00
Length: 170, dtype: float64


In [ ]:
df1.head()

,gender,idg,condtn,wave,round,position,positin1,order,partner,pid,...,attr2_3,sinc2_3,intel2_3,fun2_3,amb2_3,attr3_3,sinc3_3,intel3_3,fun3_3,amb3_3
0,0,3,2,14,18,2,2.0,14,12,372.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,14,1,3,10,2,NaN,8,8,63.0,...,NaN,NaN,NaN,NaN,NaN,6.0,8.0,8.0,7.0,8.0
2,1,14,1,13,10,8,8.0,10,10,331.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,38,2,9,20,18,13.0,6,7,200.0,...,8.0,8.0,8.0,8.0,8.0,8.0,9.0,8.0,8.0,6.0
4,1,24,2,14,20,6,6.0,20,17,357.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


From  the info of data frame we have 8 objects so we can convert them to category to be able to apply one hot encoding later on.

In [ ]:
df2 = df1.copy()




# categorical encoding of field
df2['field'] = df1['field'].astype("category")

# categorical encoding of undergra
df2['undergra'] = df1['undergra'].astype("category")

# categorical encoding of mn_sat
df2['mn_sat'] = df1['mn_sat'].astype("category")
# categorical encoding of tuition
df2['tuition'] = df1['tuition'].astype("category")
# categorical encoding of from
df2['from'] = df1['from'].astype("category")
# categorical encoding of zibcode
df2['zipcode'] = df1['zipcode'].astype("category")
# categorical encoding of income
df2['income'] = df1['income'].astype("category")
df2['career'] = df1['career'].astype("category")


In [ ]:
# now we can split the data
import numpy as np
from sklearn.model_selection import train_test_split

y = df2['match'] #target
X = df2.drop('match', axis=1) # features
print('original shape', X.shape, y.shape)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42)



original shape (5909, 169) (5909,)


manipulating

In [ ]:
df.describe()

,gender,idg,condtn,wave,round,position,positin1,order,partner,pid,...,sinc3_3,intel3_3,fun3_3,amb3_3,attr5_3,sinc5_3,intel5_3,fun5_3,amb5_3,id
count,5909.000000,5909.000000,5909.000000,5909.000000,5909.000000,5909.000000,4591.000000,5909.00000,5909.000000,5901.000000,...,2804.000000,2804.000000,2804.000000,2804.000000,1413.000000,1413.000000,1413.000000,1413.000000,1413.000000,5909.000000
mean,0.505331,17.360298,1.824843,11.347436,16.850228,9.001523,9.254846,8.91166,8.962938,283.733266,...,8.105563,8.377318,7.644437,7.398716,6.799717,7.631989,7.944798,7.162774,7.092711,4191.314943
std,0.500014,10.947542,0.380133,6.011495,4.389246,5.482368,5.611803,5.45710,5.500706,158.993002,...,1.601011,1.459013,1.757559,1.956924,1.535768,1.498024,1.320919,1.687431,1.713729,2408.009173
min,0.000000,1.000000,1.000000,1.000000,5.000000,1.000000,1.000000,1.00000,1.000000,1.000000,...,2.000000,3.000000,2.000000,1.000000,2.000000,2.000000,4.000000,1.000000,1.000000,0.000000
25%,0.000000,8.000000,2.000000,7.000000,14.000000,4.000000,4.000000,4.00000,4.000000,153.000000,...,7.000000,8.000000,7.000000,6.000000,6.000000,7.000000,7.000000,6.000000,6.000000,2124.000000
50%,1.000000,16.000000,2.000000,11.000000,18.000000,8.000000,9.000000,8.00000,8.000000,280.000000,...,8.000000,8.000000,8.000000,8.000000,7.000000,8.000000,8.000000,7.000000,7.000000,4210.000000
75%,1.000000,26.000000,2.000000,15.000000,20.000000,13.000000,14.000000,13.00000,13.000000,409.000000,...,9.000000,9.000000,9.000000,9.000000,8.000000,9.000000,9.000000,8.000000,8.000000,6266.000000
max,1.000000,44.000000,2.000000,21.000000,22.000000,22.000000,22.000000,22.00000,22.000000,552.000000,...,12.000000,12.000000,12.000000,12.000000,10.000000,10.000000,10.000000,10.000000,10.000000,8372.000000


In [ ]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5909 entries, 0 to 5908
Columns: 170 entries, gender to amb3_3
dtypes: category(8), float64(152), int64(10)
memory usage: 7.5 MB


In [ ]:
# we extract numeric features and categorical features names
# for later use

# numeric features can be selected by: (based on the df2.info() output )
features_numeric = list(X_train.select_dtypes(include=['float64', 'int64']))

# categorical features can be selected by: (based on the df2.info() output )
features_categorical = list(X_train.select_dtypes(include=['category']))

print('numeric features:', features_numeric)


numeric features: ['gender', 'idg', 'condtn', 'wave', 'round', 'position', 'positin1', 'order', 'partner', 'pid', 'int_corr', 'samerace', 'age_o', 'race_o', 'pf_o_att', 'pf_o_sin', 'pf_o_int', 'pf_o_fun', 'pf_o_amb', 'pf_o_sha', 'attr_o', 'sinc_o', 'intel_o', 'fun_o', 'amb_o', 'shar_o', 'like_o', 'prob_o', 'met_o', 'age', 'field_cd', 'race', 'imprace', 'imprelig', 'goal', 'date', 'go_out', 'career_c', 'sports', 'tvsports', 'exercise', 'dining', 'museums', 'art', 'hiking', 'gaming', 'clubbing', 'reading', 'tv', 'theater', 'movies', 'concerts', 'music', 'shopping', 'yoga', 'exphappy', 'attr1_1', 'sinc1_1', 'intel1_1', 'fun1_1', 'amb1_1', 'shar1_1', 'attr4_1', 'sinc4_1', 'intel4_1', 'fun4_1', 'amb4_1', 'shar4_1', 'attr2_1', 'sinc2_1', 'intel2_1', 'fun2_1', 'amb2_1', 'shar2_1', 'attr3_1', 'sinc3_1', 'fun3_1', 'intel3_1', 'amb3_1', 'attr5_1', 'sinc5_1', 'intel5_1', 'fun5_1', 'amb5_1', 'attr', 'sinc', 'intel', 'fun', 'amb', 'shar', 'like', 'prob', 'met', 'match_es', 'attr1_s', 'sinc1_s', 'in

In [ ]:
print('object features', features_categorical)

object features ['field', 'undergra', 'mn_sat', 'tuition', 'from', 'zipcode', 'income', 'career']


load libraries

In [ ]:

import numpy as np
#from sklearn .metrics import roc_auc_score
 
from sklearn.compose import ColumnTransformer
from sklearn.datasets import fetch_openml
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from xgboost.sklearn import XGBClassifier
from sklearn import metrics
#from sklearn.feature_selection import RFE
from sklearn.feature_selection import SelectKBest, mutual_info_classif

define pipline

In [ ]:
# define a pipe line for numeric feature preprocessing
# we gave them a name so we can set their hyperparameters
transformer_numeric = Pipeline(
    steps=[
        ('imputer', SimpleImputer()),
        ('scaler', StandardScaler())]
)

# define a pipe line for categorical feature preprocessing
# we gave them a name so we can set their hyperparameters
transformer_categorical = Pipeline(
    steps=[
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('onehot', OneHotEncoder(handle_unknown='ignore'))
    ]
)
# define the preprocessor 
# we gave them a name so we can set their hyperparameters
# we also specify what are the categorical 
preprocessor = ColumnTransformer(
    transformers=[
        ('num', transformer_numeric, features_numeric),
        ('cat', transformer_categorical,features_categorical )
    ]
)


 # Build full pipline

In [ ]:
# combine the preprocessor with the model as a full tunable pipeline
# we gave them a name so we can set their hyperparameters
full_pipline = Pipeline(
    steps=[
        ('preprocessor', preprocessor),
        
         ('selector', SelectKBest(mutual_info_classif, k=120)),
         ('my_classifier',  RandomForestClassifier( random_state=42)),
    
      
    ]
)
full_pipline

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['gender', 'idg', 'condtn',
                                                   'wave', 'round', 'position',
                                                   'positin1', 'order',
                                                   'partner', 'pid', 'int_corr',
                                                   'samerace', 'age_o',
                                                   'race_o', 'pf_o_att',
                                                   'pf_o_sin', 'pf_o_int',
                                                   

fit model

In [ ]:
full_pipline = full_pipline.fit(X_train, y_train)

y_pred=full_pipline.predict_proba(X_test)# predicton on x_test
y_pred
# roc_auc_score(y_test, y_pred)
# auc = metrics.roc_auc_score(y_train, y_pred)
# y_train.shape
# y_pred.shape

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/cluster/_supervised.py:65: UserWarning: Clustering metrics expects discrete values but received continuous values for label, and binary values for target
  warnings.warn(msg, UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/cluster/_supervised.py:65: UserWarning: Clustering metrics expects discrete values but received continuous values for label, and binary values for target
  warnings.warn(msg, UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/cluster/_supervised.py:65: UserWarning: Clustering metrics expects discrete values but received continuous values for label, and binary values for target
  warnings.warn(msg, UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/cluster/_supervised.py:65: UserWarning: Clustering metrics expects discrete values but received continuous values for label, and binary values for target
  warnings.warn(msg, UserWarning)
/usr/local/lib/python3.7/dis

array([[0.95, 0.05],
       [1.  , 0.  ],
       [0.9 , 0.1 ],
       ...,
       [0.42, 0.58],
       [0.68, 0.32],
       [0.49, 0.51]])

#Read test data

In [ ]:
test_test = pd.read_csv("/content/test.csv")

In [ ]:
test_test.head()

,gender,idg,condtn,wave,round,position,positin1,order,partner,pid,...,sinc3_3,intel3_3,fun3_3,amb3_3,attr5_3,sinc5_3,intel5_3,fun5_3,amb5_3,id
0,0,5,2,2,16,3,NaN,13,13,52.0,...,7.0,8.0,6.0,8.0,NaN,NaN,NaN,NaN,NaN,934
1,0,33,2,14,18,6,6.0,4,8,368.0,...,8.0,7.0,7.0,8.0,6.0,7.0,6.0,5.0,5.0,6539
2,1,6,2,9,20,10,16.0,15,19,212.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6757
3,1,26,2,2,19,15,NaN,8,10,30.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2275
4,0,29,2,7,16,7,7.0,10,5,162.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1052


From  the info of data frame we have 8 objects so we can convert them to category to be able to apply one hot encoding later on.

In [ ]:
df2_ts = test_test.copy()

# use the length of the name instead


# categorical encoding of field
df2_ts['field'] = test_test['field'].astype("category")

# categorical encoding of undergra
df2_ts['undergra'] = test_test['undergra'].astype("category")

# categorical encoding of mn_sat
df2_ts['mn_sat'] = test_test['mn_sat'].astype("category")
# categorical encoding of tuition
df2_ts['tuition'] = test_test['tuition'].astype("category")
# categorical encoding of from
df2_ts['from'] = test_test['from'].astype("category")
# categorical encoding of zibcode
df2_ts['zipcode'] = test_test['zipcode'].astype("category")
# categorical encoding of income
df2_ts['income'] = test_test['income'].astype("category")# categorical encoding of undergra
# categorical encoding of career
df2_ts['career'] = test_test['career'].astype("category")


In [ ]:
df2_ts = df2_ts.drop('id', axis=1)

In [ ]:
df2_ts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2469 entries, 0 to 2468
Columns: 190 entries, gender to amb5_3
dtypes: category(8), float64(173), int64(9)
memory usage: 3.5 MB


fit model

In [ ]:

full_pipline = full_pipline.fit(X, y)

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/cluster/_supervised.py:65: UserWarning: Clustering metrics expects discrete values but received continuous values for label, and binary values for target
  warnings.warn(msg, UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/cluster/_supervised.py:65: UserWarning: Clustering metrics expects discrete values but received continuous values for label, and binary values for target
  warnings.warn(msg, UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/cluster/_supervised.py:65: UserWarning: Clustering metrics expects discrete values but received continuous values for label, and binary values for target
  warnings.warn(msg, UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/cluster/_supervised.py:65: UserWarning: Clustering metrics expects discrete values but received continuous values for label, and binary values for target
  warnings.warn(msg, UserWarning)
/usr/local/lib/python3.7/dis

In [ ]:
test_pred=full_pipline.predict_proba(df2_ts)# prediction on all dataset

In [ ]:
test_pred

array([[0.85, 0.15],
       [0.65, 0.35],
       [0.85, 0.15],
       ...,
       [0.87, 0.13],
       [0.87, 0.13],
       [0.9 , 0.1 ]])

SAve result

In [ ]:
submission = pd.DataFrame()

submission['id'] = test_test['id']

submission['match'] = full_pipline.predict_proba(df2_ts)[:,1]

submission.to_csv('try2 fs1.csv', index=False)

Try b: grid search with validation set

In [ ]:
param_grid = {
    'preprocessor__num__imputer__strategy': ['most_frequent'],
    # preprocessor__num__imputer__strategy points to preprocessor->num (a Pipeline)-> imputer -> strategy
    'my_classifier__n_estimators': [90,100,120],  
     # my_classifier__n_estimators points to my_classifier->n_estimators 
    'my_classifier__max_depth':[10, 20, 30]       
}

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.model_selection import PredefinedSplit

# Further split the original training set to a train and a validation set
X_train2, X_val, y_train2, y_val = train_test_split(
    X_train, y_train, train_size = 0.8, stratify = y_train, random_state = 2022)

# Create a list where train data indices are -1 and validation data indices are 0
# X_train2 (new training set), X_train
split_index = [-1 if x in X_train2.index else 0 for x in X_train.index]

# Use the list to create PredefinedSplit
pds = PredefinedSplit(test_fold = split_index)

grid_search = GridSearchCV(
    full_pipline, param_grid, cv=5, verbose=1, n_jobs=2, 
    scoring='roc_auc')

grid_search.fit(X_train, y_train)

print('best score {}'.format(grid_search.best_score_))
print('best score {}'.format(grid_search.best_params_))


Fitting 5 folds for each of 9 candidates, totalling 45 fits


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/cluster/_supervised.py:65: UserWarning: Clustering metrics expects discrete values but received continuous values for label, and binary values for target
  warnings.warn(msg, UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/cluster/_supervised.py:65: UserWarning: Clustering metrics expects discrete values but received continuous values for label, and binary values for target
  warnings.warn(msg, UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/cluster/_supervised.py:65: UserWarning: Clustering metrics expects discrete values but received continuous values for label, and binary values for target
  warnings.warn(msg, UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/cluster/_supervised.py:65: UserWarning: Clustering metrics expects discrete values but received continuous values for label, and binary values for target
  warnings.warn(msg, UserWarning)
/usr/local/lib/python3.7/dis

best score 0.8495876661455606
best score {'my_classifier__max_depth': 10, 'my_classifier__n_estimators': 120, 'preprocessor__num__imputer__strategy': 'most_frequent'}


In [ ]:
grid_search.predict_proba(X_test)# prediction on X_test

array([[0.95776712, 0.04223288],
       [0.98700541, 0.01299459],
       [0.86372175, 0.13627825],
       ...,
       [0.50075059, 0.49924941],
       [0.74203527, 0.25796473],
       [0.4756211 , 0.5243789 ]])

In [ ]:
full_pipline = full_pipline.fit(X, y) #fit model on all data.

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/cluster/_supervised.py:65: UserWarning: Clustering metrics expects discrete values but received continuous values for label, and binary values for target
  warnings.warn(msg, UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/cluster/_supervised.py:65: UserWarning: Clustering metrics expects discrete values but received continuous values for label, and binary values for target
  warnings.warn(msg, UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/cluster/_supervised.py:65: UserWarning: Clustering metrics expects discrete values but received continuous values for label, and binary values for target
  warnings.warn(msg, UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/cluster/_supervised.py:65: UserWarning: Clustering metrics expects discrete values but received continuous values for label, and binary values for target
  warnings.warn(msg, UserWarning)
/usr/local/lib/python3.7/dis

In [ ]:
test_pred=full_pipline.predict_proba(df2_ts)# prediction of all data set
test_pred

array([[0.85, 0.15],
       [0.65, 0.35],
       [0.85, 0.15],
       ...,
       [0.87, 0.13],
       [0.87, 0.13],
       [0.9 , 0.1 ]])

Save result

In [ ]:
submission = pd.DataFrame()

submission['id'] = test_test['id']

submission['match'] = full_pipline.predict_proba(df2_ts)[:,1]

submission.to_csv('try2 fs2.csv', index=False)

Tryc: Random search

In [ ]:
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.model_selection import PredefinedSplit

grid_search = RandomizedSearchCV(
    full_pipline, param_grid, cv=pds, verbose=1, n_jobs=2, 
    # number of random trials
    n_iter=10,
    scoring='roc_auc')

grid_search.fit(X_train, y_train)

print('best score {}'.format(grid_search.best_score_))
print('best score {}'.format(grid_search.best_params_))

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_search.py:296: UserWarning: The total space of parameters 9 is smaller than n_iter=10. Running 9 iterations. For exhaustive searches, use GridSearchCV.
  UserWarning,


Fitting 1 folds for each of 9 candidates, totalling 9 fits


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/cluster/_supervised.py:65: UserWarning: Clustering metrics expects discrete values but received continuous values for label, and binary values for target
  warnings.warn(msg, UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/cluster/_supervised.py:65: UserWarning: Clustering metrics expects discrete values but received continuous values for label, and binary values for target
  warnings.warn(msg, UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/cluster/_supervised.py:65: UserWarning: Clustering metrics expects discrete values but received continuous values for label, and binary values for target
  warnings.warn(msg, UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/cluster/_supervised.py:65: UserWarning: Clustering metrics expects discrete values but received continuous values for label, and binary values for target
  warnings.warn(msg, UserWarning)
/usr/local/lib/python3.7/dis

best score 0.862399233327696
best score {'preprocessor__num__imputer__strategy': 'most_frequent', 'my_classifier__n_estimators': 100, 'my_classifier__max_depth': 20}


In [ ]:
grid_search.predict_proba(X_test)# prediction on x_test

array([[0.97984816, 0.02015184],
       [0.99      , 0.01      ],
       [0.8098827 , 0.1901173 ],
       ...,
       [0.41      , 0.59      ],
       [0.71      , 0.29      ],
       [0.39584682, 0.60415318]])

In [ ]:
full_pipline = full_pipline.fit(X, y)# fit model on all data

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/cluster/_supervised.py:65: UserWarning: Clustering metrics expects discrete values but received continuous values for label, and binary values for target
  warnings.warn(msg, UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/cluster/_supervised.py:65: UserWarning: Clustering metrics expects discrete values but received continuous values for label, and binary values for target
  warnings.warn(msg, UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/cluster/_supervised.py:65: UserWarning: Clustering metrics expects discrete values but received continuous values for label, and binary values for target
  warnings.warn(msg, UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/cluster/_supervised.py:65: UserWarning: Clustering metrics expects discrete values but received continuous values for label, and binary values for target
  warnings.warn(msg, UserWarning)
/usr/local/lib/python3.7/dis

In [ ]:
test_pred=full_pipline.predict_proba(df2_ts)# prediction on all data set.
test_pred

array([[0.85, 0.15],
       [0.65, 0.35],
       [0.85, 0.15],
       ...,
       [0.87, 0.13],
       [0.87, 0.13],
       [0.9 , 0.1 ]])

Saving result

In [ ]:
submission = pd.DataFrame()

submission['id'] = test_test['id']

submission['match'] = full_pipline.predict_proba(df2_ts)[:,1]

submission.to_csv('try311afaf.csv', index=False)

In [ ]:
!pip install scikit_optimize

Try d baysian

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import PredefinedSplit
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer



bayes_search = BayesSearchCV(
    full_pipline, param_grid, cv=pds, verbose=1, n_jobs=2, 
    # number of random trials
    n_iter=10,
    scoring='roc_auc')

bayes_search.fit(X_train, y_train)

print('best score {}'.format(bayes_search.best_score_))
print('best score {}'.format(bayes_search.best_params_))

Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/cluster/_supervised.py:65: UserWarning: Clustering metrics expects discrete values but received continuous values for label, and binary values for target
  warnings.warn(msg, UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/cluster/_supervised.py:65: UserWarning: Clustering metrics expects discrete values but received continuous values for label, and binary values for target
  warnings.warn(msg, UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/cluster/_supervised.py:65: UserWarning: Clustering metrics expects discrete values but received continuous values for label, and binary values for target
  warnings.warn(msg, UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/cluster/_supervised.py:65: UserWarning: Clustering metrics expects discrete values but received continuous values for label, and binary values for target
  warnings.warn(msg, UserWarning)
/usr/local/lib/python3.7/dis

best score 0.862399233327696
best score OrderedDict([('my_classifier__max_depth', 20), ('my_classifier__n_estimators', 100), ('preprocessor__num__imputer__strategy', 'most_frequent')])


In [ ]:
bayes_search.predict_proba(X_test)# prediction on x_test

array([[0.97984816, 0.02015184],
       [0.99      , 0.01      ],
       [0.8098827 , 0.1901173 ],
       ...,
       [0.41      , 0.59      ],
       [0.71      , 0.29      ],
       [0.39584682, 0.60415318]])

In [ ]:
full_pipline = full_pipline.fit(X, y)# fit model on all data.

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/cluster/_supervised.py:65: UserWarning: Clustering metrics expects discrete values but received continuous values for label, and binary values for target
  warnings.warn(msg, UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/cluster/_supervised.py:65: UserWarning: Clustering metrics expects discrete values but received continuous values for label, and binary values for target
  warnings.warn(msg, UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/cluster/_supervised.py:65: UserWarning: Clustering metrics expects discrete values but received continuous values for label, and binary values for target
  warnings.warn(msg, UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/cluster/_supervised.py:65: UserWarning: Clustering metrics expects discrete values but received continuous values for label, and binary values for target
  warnings.warn(msg, UserWarning)
/usr/local/lib/python3.7/dis

In [ ]:
test_pred=full_pipline.predict_proba(df2_ts)#prediction on all data
test_pred

array([[0.85, 0.15],
       [0.65, 0.35],
       [0.85, 0.15],
       ...,
       [0.87, 0.13],
       [0.87, 0.13],
       [0.9 , 0.1 ]])

Saving result

In [ ]:
submission = pd.DataFrame() 

submission['id'] = test_test['id']

submission['match'] = full_pipline.predict_proba(df2_ts)[:,1]

submission.to_csv('try2 fs bayesafaf.csv', index=False)

## Questions

Why a simple linear regression model (without any activation function) is not good for classification task, compared to Perceptron/Logistic regression?

* **Logistic Regression** deals with discrete values unlike **Linear Regression** and also maintains the value of the threshold even when new data points are added.
 **Linear Regression** deals with continuous values whereas classification problems mandate discrete values.
 The second problem is regarding the shift in threshold value when new data points are added.

What's a decision tree and how it is different to a logistic regression model?

**A decision tree** is a type of supervised machine learning
 used to categorize or make predictions based on how a previous set of questions were answered. 
 * Decision trees come from Decision Science. Their purpose is the plot out a series of sequential decisions, their dependents, and the ultimate outcome from that set of decisions.

**Logistic Regression** comes out of Statistics. It is a tool used to estimate the marginal impact a set of independent predictor variables have upon the likelihood of an outcome.




What's the difference between grid search and random search?

Two of them are used for tunning hyperparamters.
**In Grid Search**, we try every combination of a preset list of values of the hyper-parameters and choose the best combination based on the cross validation score.

**Random searc**h tries random combinations of a range of values (we have to define the number iterations). 
It is good in testing a wide range of values and normally it reaches a very good combination very fast,
 but the problem that it doesn’t guarantee to give the best parameters combination.

On the other hand, **Grid search** will give the best combination but it can take a lot of time.

What's the difference between bayesian search and random search?

Two of them are used for tunning hyperparamters.
**Bayesian optimization** methods are efficient because they select hyperparameters in an informed manner.
 By prioritizing hyperparameters that appear more promising from past results.
 

* •As an optimization problem
* •Trial -> estimated error -> Bayesian model estimates the next 
* parameter to try -> trial -> repeat..


* **Random search:**
Try out a random subset 
* •`good enough`
* •Local optimal (within the given range)
* •Efficient (less trials)
* the best of them
* **Bayesian methods** can find the best hyperparameters in lesser time (in fewer iterations)  than random search





